# **Descriptive Statistics of Participants and Survey Responses**
Lindsay Warrenburg & Lauren Fink  
lindsay.a.warrenburg@gmail.com  lauren.fink@ae.mpg.de  
Final version: December 20, 2020

### This script describes basic summary statistics of participants and the major sections of the survey

- No statistical tests are conducted
- Descriptives primarily consist of means, standard deviations, and percent changes
- Country-level differences are reported for the major questionnaire sections: (1) Activities during lockdown, (2) Coping-related behaviors, (3) Functions of music listening/music making, (4) Situations of music listening/music making, (5) Music listening formats, (6) Kinds of music engagement

In [2]:
import numpy as np, pandas as pd, os, sys, itertools, re, warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings(action='ignore')
%run Coronavirus_Music_Functions.ipynb

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.shape

(5113, 234)

# Participant Information

In [4]:
demographics = [col for col in data.columns if 'Demographics_' in col]

### Country Counts

In [7]:
pd.DataFrame(data['Country_Country Name'].value_counts(dropna=False)).sort_index()

5113


,Country_Country Name
France,983
Germany,872
India,891
Italy,892
UK,621
USA,854


### Basic Info

In [7]:
# number of times leave the house per week, age, education, musicianship
group_col = 'Country_Country Name'
col_list = ['Demographics_COVID_Number Times Leave','Demographics_General_Age', 
            'Demographics_General_Education Years', 'Demographics_Music_Ollen']

tmp = group_numeric(data, col_list, group_col, 0)

tmp.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Demographics_COVID_Number Times Leave_mean,3.21,6.74,3.18,2.91,3.23,3.23,3.76
Demographics_General_Age_mean,49.51,45.86,37.25,51.18,32.13,45.25,44.22
Demographics_General_Education Years_mean,17.64,15.24,12.37,12.63,16.48,16.57,15.12
Demographics_Music_Ollen_mean,1.91,2.03,2.90,2.08,2.03,2.11,2.17
Demographics_COVID_Number Times Leave_sd,3.56,5.71,4.81,3.71,3.22,3.68,4.44
Demographics_General_Age_sd,17.25,12.97,13.67,17.06,11.47,14.51,16.15
Demographics_General_Education Years_sd,5.22,5.94,6.84,5.30,3.63,5.50,5.93
Demographics_Music_Ollen_sd,0.78,0.81,1.64,0.76,0.99,1.18,1.11


In [10]:
# gender 
pd.DataFrame(data.groupby(['Country_Country Name'])['Demographics_General_Gender'].value_counts(dropna=False, normalize=True)).sort_index().round(2).T

Country_Country Name        France                 Germany                  \
Demographics_General_Gender    1.0   2.0  4.0  NaN     1.0   2.0  3.0  4.0   
Demographics_General_Gender   0.52  0.48  0.0  0.0    0.52  0.48  0.0  0.0   

Country_Country Name        India             Italy               UK        \
Demographics_General_Gender   1.0   2.0   4.0   1.0   2.0  NaN   1.0   2.0   
Demographics_General_Gender  0.44  0.56  0.01  0.53  0.47  0.0  0.71  0.28   

Country_Country Name                     USA             
Demographics_General_Gender  3.0   4.0   1.0   2.0  4.0  
Demographics_General_Gender  0.0  0.01  0.57  0.43  0.0

### Infected with the COVID-19 Virus

In [10]:
col_list = ['Demographics_Health_COVID Contact Binary']+[c for c in data.columns if 'Infected with COVID' in c]
group_col = 'Country_Country Name'

tmp = temp_df(group_col, col_list, data)
grouped_n, grouped_perc = make_grouped_df(group_col, data, tmp)
grouped_perc.transpose()

group,France,Germany,India,Italy,UK,USA,TOTAL
cols,,,,,,,
Demographics_Health_COVID Contact Binary,26.45,10.09,28.62,16.03,23.35,33.14,22.96
I Am Infected,2.75,1.83,8.42,1.12,4.51,4.80,3.85
I Am Mourning A Loss,2.64,0.57,6.17,3.92,2.58,6.32,3.74
I Have Professional Contact,6.51,3.44,10.89,2.24,3.86,6.09,5.61
I Live With Someone,2.34,1.38,10.21,0.56,2.09,4.45,3.56
No Response,4.27,1.72,68.91,2.02,75.52,64.52,33.42
None,61.75,87.04,0.00,77.80,0.00,0.00,40.29
Prefer Not To Say,8.14,1.72,2.47,4.93,1.13,2.34,3.68
Someone I Am Close To,16.79,4.47,10.44,11.10,14.65,22.37,13.26


### Living Situation During COVID-19

In [11]:
col_list = [c for c in data.columns if c.startswith('Demographics_COVID_Living Situation')]
group_col = 'Country_Country Name'

tmp = temp_df(group_col, col_list, data) 
grouped_n, grouped_perc = make_grouped_df(group_col, data, tmp)
grouped_perc.transpose()

group,France,Germany,India,Italy,UK,USA,TOTAL
cols,,,,,,,
Alone,20.55,25.23,7.30,10.54,9.98,17.10,15.43
Child,30.62,26.83,44.11,23.88,28.02,33.96,31.39
Elderly,1.12,0.34,2.58,1.68,2.25,2.34,1.68
Friend,0.20,1.03,2.36,0.56,5.15,2.22,1.72
Other,0.92,0.57,1.23,6.73,0.48,0.94,1.88
Parent,7.93,9.75,43.77,16.03,26.57,15.34,19.40
Partner,62.26,56.19,55.78,63.68,49.28,56.56,57.81
Pet,25.33,20.76,7.97,19.62,19.97,30.44,20.73
Shared,0.41,1.83,2.13,1.12,5.80,2.22,2.03


### Ways Work is Affected Due to COVID-19

In [12]:
col_list = ['Demographics_COVID_Work Changed Binary']+[c for c in data.columns if c.startswith('Demographics_COVID_Ways Work Affected')]
group_col = 'Country_Country Name'

tmp = temp_df(group_col, col_list, data)
grouped_n, grouped_perc = make_grouped_df(group_col, data, tmp)
grouped_perc.transpose()

group,France,Germany,India,Italy,UK,USA,TOTAL
cols,,,,,,,
Demographics_COVID_Work Changed Binary,39.37,41.40,73.51,39.57,58.78,59.25,51.38
Homeschool Children,11.70,7.34,7.41,17.94,14.01,6.44,10.70
Job At Risk,1.32,5.85,16.16,5.72,5.48,5.85,6.71
Lost Job,2.75,4.59,16.50,8.18,9.34,11.12,8.61
Lower Income,11.39,15.94,30.30,16.48,19.32,15.93,18.07
Now Work From Home,18.41,16.28,39.28,9.42,30.76,30.80,23.68
Reduced Hours,11.70,11.01,23.79,10.87,10.47,16.51,14.20
Still Go To Work,32.96,42.32,13.58,23.77,17.87,21.31,25.80
Still Work From Home,22.18,17.43,25.59,24.22,19.32,21.78,21.90


### Change in Emotions due to COVID-19

In [13]:
col_list = ['Demographics_Health_General Health', 'Demographics_Health_Arousal',
            'Demographics_Health_Negative Valence','Demographics_Health_Positive Valence',
            'Demographics_Health_Anxious','Demographics_Health_Bored',
            'Demographics_Health_Depressed','Demographics_Health_Lonely',
            'Demographics_Health_Stressed']

group_col = 'Country_Country Name'
new = group_numeric(data, col_list, group_col, 1)

new.columns = new.columns.str.split('Health_').str[1]
new.T.round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Anxious_change,2.08,8.71,20.21,15.41,30.32,26.63,16.26
Anxious_mean,4.06,4.26,4.61,4.46,4.91,4.80,4.49
Anxious_sd,1.58,1.19,1.70,1.42,1.36,1.32,1.47
Arousal_change,-17.19,-9.04,21.51,-20.90,-27.10,-10.17,-10.08
Arousal_mean,3.48,3.73,4.65,3.37,3.19,3.69,3.70
Arousal_sd,1.41,1.09,1.63,1.26,1.22,1.31,1.41
Bored_change,2.21,10.37,NaN,17.93,NaN,NaN,9.98
Bored_mean,4.07,4.31,NaN,4.54,NaN,NaN,4.30
Bored_sd,1.73,1.37,NaN,1.58,NaN,NaN,1.58
Depressed_change,-17.80,10.85,9.93,-2.75,19.02,15.44,4.64


### Comparing Changes in Emotions with Employment Status

In [14]:
# mean difference in emotion scores between people who said 'yes' and 'no' to the following work situations
col_list = ['Demographics_Health_General Health', 'Demographics_Health_Arousal',
            'Demographics_Health_Negative Valence','Demographics_Health_Positive Valence',
            'Demographics_Health_Anxious','Demographics_Health_Bored',
            'Demographics_Health_Depressed','Demographics_Health_Lonely',
            'Demographics_Health_Stressed']

index_list = [col for col in demographics if col.startswith('Demographics_COVID_Ways Work Affected') and
              'No Response' not in col]

yes_minus_no, perc_diff = mean_differences(data, index_list, col_list)

yes_minus_no.columns = yes_minus_no.columns.str.split('Health_').str[1]
perc_diff.columns = perc_diff.columns.str.split('Health_').str[1]

yes_minus_no.index = yes_minus_no.index.str.split('-').str[1]
perc_diff.index = perc_diff.index.str.split('-').str[1]

yes_minus_no.round(2)

,General Health,Arousal,Negative Valence,Positive Valence,Anxious,Bored,Depressed,Lonely,Stressed
index,,,,,,,,,
Homeschool Children,-0.134337,-0.183751,0.0441929,-0.218432,0.0905374,-0.0340734,-0.0611161,-0.192632,0.017701
Job At Risk,0.162568,0.0693702,0.458982,0.145759,0.561566,0.241675,0.588852,0.557144,0.559406
Lost Job,0.17711,0.147189,0.353928,0.196187,0.460997,0.417742,0.517345,0.329387,0.483363
Lower Income,0.0860386,-0.00213847,0.256062,0.0595841,0.359494,0.112081,0.45823,0.375862,0.346011
Now Work From Home,0.273984,0.300384,0.0441438,0.327125,0.179243,0.213053,0.2041,0.192715,0.186907
Reduced Hours,0.280035,0.402667,0.119851,0.454863,0.21587,0.111486,0.369151,0.260358,0.182768
Still Go To Work,-0.0110763,0.0942662,-0.123275,0.0449092,-0.240579,-0.226523,-0.2006,-0.152434,-0.123355
Still Work From Home,0.0858585,0.185412,-0.0567193,0.195583,-0.0428414,-0.152958,-0.0541715,0.0104104,-0.0834444


In [15]:
# percent difference in emotion scores between people who answer 'yes' and 'no' to the employment questions
perc_diff.round(2)

,General Health,Arousal,Negative Valence,Positive Valence,Anxious,Bored,Depressed,Lonely,Stressed
index,,,,,,,,,
Homeschool Children,-1.91911,-2.62501,0.631327,-3.12045,1.29339,-0.486763,-0.873088,-2.75189,0.252871
Job At Risk,2.32241,0.991003,6.55689,2.08227,8.02237,3.4525,8.41217,7.95919,7.99152
Lost Job,2.53014,2.1027,5.05611,2.80268,6.58567,5.96775,7.39064,4.70552,6.90519
Lower Income,1.22912,-0.0305496,3.65802,0.851201,5.13563,1.60116,6.54614,5.36946,4.94301
Now Work From Home,3.91406,4.29119,0.630626,4.67321,2.56062,3.04362,2.91572,2.75307,2.6701
Reduced Hours,4.0005,5.75238,1.71216,6.49804,3.08386,1.59266,5.27358,3.7194,2.61096
Still Go To Work,-0.158233,1.34666,-1.76107,0.64156,-3.43684,-3.23605,-2.86572,-2.17762,-1.76222
Still Work From Home,1.22655,2.64875,-0.810276,2.79404,-0.61202,-2.18511,-0.773879,0.14872,-1.19206


# Major Survey Sections

### Activities

In [16]:
col_list = [col for col in data.columns if col.startswith('Activities_')]
group_col = 'Country_Country Name'

new = group_numeric(data, col_list, group_col, 1)
new.columns = new.columns.str.split('Activities_').str[1]
new.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Call People_change,31.20,22.91,46.35,26.46,55.25,46.14,37.03
Call People_mean,4.94,4.69,5.39,4.79,5.66,5.38,5.11
Call People_sd,1.45,1.27,1.72,1.57,1.32,1.38,1.50
Cleaning_change,28.66,14.93,53.06,29.21,39.97,38.73,33.64
Cleaning_mean,4.86,4.45,5.59,4.88,5.20,5.16,5.01
Cleaning_sd,1.33,1.05,1.62,1.40,1.34,1.41,1.41
Cooking_change,25.63,17.29,41.27,22.30,33.93,37.42,29.28
Cooking_mean,4.77,4.52,5.24,4.67,5.02,5.12,4.88
Cooking_sd,1.31,1.07,1.61,1.42,1.27,1.36,1.38
Crafting_change,16.18,8.95,19.61,13.97,11.09,14.78,14.25


### Coping Behaviors

In [17]:
# music listening
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Music Listening_Hypotheses')]
listening = group_numeric(data, col_list, group_col, 0)
listening.columns = listening.columns.str.split('Hypotheses_').str[1]

listening.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Feel Connected to Others_mean,3.68,3.56,5.50,4.03,4.14,4.55,4.24
Helps Me Cope_mean,4.29,3.87,5.58,4.86,4.93,4.94,4.73
Serve as Social Interaction_mean,3.62,3.52,5.48,3.80,4.09,4.52,4.17
Feel Connected to Others_sd,1.54,1.72,1.40,1.70,1.65,1.54,1.72
Helps Me Cope_sd,1.55,1.79,1.38,1.67,1.47,1.45,1.65
Serve as Social Interaction_sd,1.62,1.75,1.45,1.80,1.72,1.61,1.79


In [18]:
# music making
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Making Music_Hypotheses')]
performance = group_numeric(data, col_list, group_col, 0)
performance.columns = performance.columns.str.split('Hypotheses_').str[1]

performance.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Feel Connected to Others_mean,3.79,3.56,5.39,4.19,3.83,4.36,4.35
Helps Me Cope_mean,3.80,3.80,5.56,4.76,4.53,4.64,4.66
Serve as Social Interaction_mean,4.32,3.41,5.45,3.93,3.86,4.27,4.37
Feel Connected to Others_sd,1.44,1.86,1.50,1.76,1.59,1.70,1.75
Helps Me Cope_sd,1.44,1.87,1.42,1.74,1.61,1.72,1.73
Serve as Social Interaction_sd,1.47,1.83,1.48,1.79,1.70,1.78,1.79


In [19]:
# difference between music listening and music making
mean_cols = [c for c in listening if '_mean' in c] 
listen_minus_perform = listening - performance
listen_minus_perform[mean_cols].transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Feel Connected to Others_mean,-0.11,0.00,0.10,-0.17,0.31,0.19,-0.11
Helps Me Cope_mean,0.49,0.08,0.02,0.10,0.40,0.30,0.07
Serve as Social Interaction_mean,-0.70,0.11,0.03,-0.13,0.23,0.26,-0.20


### Functions

In [20]:
# music listening
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Music Listening_Functions')]
listening = group_numeric(data, col_list, group_col, 1)
listening.columns = listening.columns.str.split('Functions_').str[1]

listening.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Creates a Personal Space_change,10.96,12.92,49.03,20.13,28.67,31.28,25.21
Creates a Personal Space_mean,4.33,4.39,5.47,4.60,4.86,4.94,4.76
Creates a Personal Space_sd,1.16,1.08,1.51,1.45,1.32,1.30,1.37
Enables Me to Vent Negative Emotions_change,20.14,14.44,46.55,32.40,22.60,25.95,27.13
Enables Me to Vent Negative Emotions_mean,4.60,4.43,5.40,4.97,4.68,4.78,4.81
Enables Me to Vent Negative Emotions_sd,1.23,1.10,1.58,1.43,1.31,1.34,1.37
Energizes Me_change,21.58,20.00,54.21,37.02,33.17,34.36,33.27
Energizes Me_mean,4.65,4.60,5.63,5.11,5.00,5.03,5.00
Energizes Me_sd,1.22,1.13,1.43,1.39,1.30,1.29,1.34
Feel Connected to My Culture_change,5.77,3.93,47.55,9.41,8.91,22.72,16.56


In [21]:
# music making
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Making Music_Functions')]
performance = group_numeric(data, col_list, group_col,1)
performance.columns = performance.columns.str.split('Functions_').str[1]

performance.transpose().round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Creates a Personal Space_change,11.90,15.86,48.51,22.94,23.01,29.88,28.73
Creates a Personal Space_mean,4.36,4.48,5.46,4.69,4.69,4.90,4.86
Creates a Personal Space_sd,1.18,1.17,1.43,1.48,1.25,1.31,1.38
Enables Me to Vent Negative Emotions_change,21.34,17.32,50.16,35.30,22.90,27.68,31.46
Enables Me to Vent Negative Emotions_mean,4.64,4.52,5.50,5.06,4.69,4.83,4.94
Enables Me to Vent Negative Emotions_sd,1.30,1.23,1.44,1.54,1.31,1.35,1.42
Energizes Me_change,25.07,21.61,53.78,41.13,24.93,31.65,35.26
Energizes Me_mean,4.75,4.65,5.61,5.23,4.75,4.95,5.06
Energizes Me_sd,1.23,1.15,1.35,1.47,1.29,1.28,1.35
Feel Connected to My Culture_change,9.84,10.82,49.22,17.09,7.02,23.33,23.12


In [22]:
# difference between music listening and music making
mean_cols = [c for c in listening if '_mean' in c] 
listen_minus_perform = listening - performance
listen_minus_perform[mean_cols].T.round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Creates a Personal Space_mean,-0.03,-0.09,0.02,-0.08,0.17,0.04,-0.11
Enables Me to Vent Negative Emotions_mean,-0.04,-0.09,-0.11,-0.09,-0.01,-0.05,-0.13
Energizes Me_mean,-0.10,-0.05,0.01,-0.12,0.25,0.08,-0.06
Feel Connected to My Culture_mean,-0.12,-0.21,-0.05,-0.23,0.06,-0.02,-0.20
Feel Less Lonely_mean,0.08,0.00,0.03,0.08,0.24,0.15,-0.01
Feel Like I Am Sharing My Experience_mean,-0.19,-0.22,-0.10,-0.22,0.03,-0.01,-0.23
Feel Like I Have Company_mean,-0.13,-0.11,-0.02,-0.21,0.09,0.13,-0.16
Feel Like I Understand Others_mean,-0.14,-0.13,-0.10,-0.20,0.07,-0.01,-0.20
Feel Like Part of a Bigger Group_mean,-0.11,-0.12,0.04,-0.09,0.06,0.07,-0.13
Gives Me New Perspectives_mean,-0.03,-0.14,-0.02,-0.08,0.30,0.14,-0.07


### Situations

In [23]:
# music listening
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Music Listening_Situations')]
listening = group_numeric(data, col_list, group_col,1)
listening.columns = listening.columns.str.split('Situations_').str[1]

listening.T.round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Alone_change,15.73,14.55,43.98,20.05,29.61,27.65,25.00
Alone_mean,4.47,4.44,5.32,4.60,4.89,4.83,4.75
Alone_sd,1.26,1.08,1.67,1.46,1.31,1.34,1.40
Dining_change,-3.77,3.02,13.90,-9.05,-2.79,5.51,1.24
Dining_mean,3.89,4.09,4.42,3.73,3.92,4.17,4.04
Dining_sd,1.01,0.82,1.80,1.25,1.04,1.21,1.26
Doing Nothing Else_change,10.80,8.30,40.35,10.83,25.12,24.92,19.76
Doing Nothing Else_mean,4.32,4.25,5.21,4.33,4.75,4.75,4.59
Doing Nothing Else_sd,1.21,0.95,1.66,1.39,1.25,1.31,1.36
Evening_change,-3.54,2.60,33.21,-4.61,7.33,15.66,8.37


In [24]:
# music making
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Making Music_Situations')]
performance = group_numeric(data, col_list, group_col, 1)
performance.columns = performance.columns.str.split('Situations_').str[1]

performance.T.round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Balconies_change,-5.21,0.51,23.51,-7.43,-1.88,9.43,6.11
Balconies_mean,3.84,4.02,4.71,3.78,3.94,4.28,4.18
Balconies_sd,0.86,0.85,1.74,1.19,0.76,1.13,1.26
Dance Alone_change,0.34,1.33,26.55,-1.72,12.04,15.57,12.09
Dance Alone_mean,4.01,4.04,4.80,3.95,4.36,4.47,4.36
Dance Alone_sd,1.12,0.97,1.76,1.43,1.20,1.29,1.41
Dance Internet_change,-6.05,-1.03,24.26,-5.80,1.94,10.03,7.00
Dance Internet_mean,3.82,3.97,4.73,3.83,4.06,4.30,4.21
Dance Internet_sd,0.79,0.82,1.70,1.07,0.91,1.11,1.24
Dance Together_change,-2.94,-2.77,21.88,-8.33,8.28,9.08,7.18


### Music Listening Formats

*Note: These questions were accidently left out of the Germany sample*

In [25]:
group_col = 'Country_Country Name'
col_list = [col for col in data.columns if col.startswith('Music Listening_Formats')]
listening = group_numeric(data, col_list, group_col, 1)
listening.columns = listening.columns.str.split('Formats_').str[1]

listening.T.round(2)

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Always Available Programs_change,-3.42,NaN,19.48,-1.33,2.30,13.11,6.10
Always Available Programs_mean,3.90,NaN,4.58,3.96,4.07,4.39,4.18
Always Available Programs_sd,0.89,NaN,1.80,1.17,1.02,1.11,1.28
Free Livestreams_change,3.30,NaN,35.79,3.73,17.02,23.12,16.41
Free Livestreams_mean,4.10,NaN,5.07,4.11,4.51,4.69,4.49
Free Livestreams_sd,1.05,NaN,1.67,1.27,1.21,1.24,1.36
Free Streaming Others Playlists_change,2.80,NaN,36.57,7.55,17.30,22.66,17.20
Free Streaming Others Playlists_mean,4.08,NaN,5.10,4.23,4.52,4.68,4.52
Free Streaming Others Playlists_sd,1.11,NaN,1.70,1.31,1.24,1.26,1.39
Free Streaming Own Playlists_change,5.17,NaN,35.47,4.98,17.35,23.75,17.20


### Music Engagment

Percent of people who report engaging with music differently

In [26]:
col_list = ['Music Engagement_Engaging With Music Differently YN']
group_col = 'Country_Country Name'

tmp = temp_df(group_col, col_list, data)
grouped_n, grouped_perc = make_grouped_df(group_col, data, tmp)
grouped_perc.T

group,France,Germany,India,Italy,UK,USA,TOTAL
cols,,,,,,,
Music Engagement_Engaging With Music Differently YN,20.85,9.98,70.37,27.47,35.91,38.52,33.56


Of the people who said 'yes' to engaging with music differently, what kinds of music are they engaging with?

In [27]:
# genres and musicans/composers
df = data[data['Music Engagement_Engaging With Music Differently YN'] == 1].reset_index(drop=True)
col_list = [c for c in df.columns if 'Music Types-' in c]
group_col = 'Country_Country Name'
tmp = temp_df(group_col, col_list, df)
grouped_n, grouped_perc = make_grouped_df(group_col, df, tmp)
grouped_perc.T

group,France,Germany,India,Italy,UK,USA,TOTAL
cols,,,,,,,
Other Genres,43.41,36.78,25.20,36.33,32.29,27.05,30.83
Other Musicians Composers,15.12,12.64,14.99,22.04,9.42,15.50,15.27
Same Musicians Composers,5.37,8.05,7.81,12.24,3.59,5.17,7.11


In [28]:
# diverse music, nostalgic music
col_list = ['Music Engagement_Diversity of Music', 'Music Engagement_Nostalgic Music']
group_col = 'Country_Country Name'
group_numeric(data, col_list, group_col, 1).transpose()

#pd.DataFrame(data['Music Engagement_Diversity of Music'].value_counts(dropna=False, normalize=True)).sort_index(ascending=False).round(4)*100
#pd.DataFrame(data['Music Engagement_Nostalgic Music'].value_counts(dropna=False, normalize=True)).sort_index(ascending=False).round(4)*100

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Music Engagement_Diversity of Music_change,1.362928,3.216374,22.661722,6.038048,14.277869,12.311558,9.717466
Music Engagement_Diversity of Music_mean,4.040888,4.096491,4.679852,4.181141,4.428336,4.369347,4.291524
Music Engagement_Diversity of Music_sd,1.097349,0.801168,1.835460,0.968801,0.930446,1.156557,1.210012
Music Engagement_Nostalgic Music_change,-0.312989,-3.600169,32.265243,5.416667,18.426843,18.063872,10.899447
Music Engagement_Nostalgic Music_mean,3.990610,3.891995,4.967957,4.162500,4.552805,4.541916,4.326983
Music Engagement_Nostalgic Music_sd,1.168488,1.349686,1.783477,1.204328,1.161226,1.291087,1.396182


In [29]:
# interest in coronamusic
col_list = ['Music Engagement_Interest in Others Music']
group_col = 'Country_Country Name'
group_numeric(data, col_list, group_col,0).T

Country_Country Name,France,Germany,India,Italy,UK,USA,TOTAL
Music Engagement_Interest in Others Music_mean,3.021436,2.682033,5.042527,4.150000,3.431856,3.900726,3.696315
Music Engagement_Interest in Others Music_sd,1.840056,1.734259,1.842163,1.912797,1.745964,1.951490,2.004477


In [31]:
pd.DataFrame(data['Music Engagement_Interest in Others Music'].value_counts(dropna=False, normalize=True)).sort_index(ascending=False).round(4)*100

,Music Engagement_Interest in Others Music
7.0,10.39
6.0,10.25
5.0,11.58
4.0,24.29
3.0,7.43
2.0,8.53
1.0,22.53
NaN,5.01
